# go-错误处理

### Error类型

error是一个内置的接口类型，这是它的定义：
```
type error interface {
	Error() string
}
```

### 自定义error

error是一个interface，所以在实现自己的包的时候，通过定义实现此接口的结构体(struct)，就可以实现自己的错误定义.

注意:
1. 函数返回自定义错误时，返回值推荐设置为error类型，而非自定义错误类型，
2. 不应预声明自定义错误类型的变量

### 错误处理

Go在错误处理上采用了与C类似的检查返回值的方式，而不是其他多数主流语言采用的异常方式，

这造成了代码编写上的一个很大的缺点:错误处理代码的冗余，对于这种情况是我们通过复用检测函数来减少类似的代码。

### 网站错误处理

Web应用日常运行中可能出现多种错误，具体如下所示：
* 数据库错误：指与访问数据库服务器或数据相关的错误。例如，以下可能出现的一些数据库错误。
    * 连接错误：这一类错误可能是数据库服务器网络断开、用户名密码不正确、或者数据库不存在。
    * 查询错误：使用的SQL非法导致错误，这样子SQL错误如果程序经过严格的测试应该可以避免。
    * 数据错误：数据库中的约束冲突，例如一个唯一字段中插入一条重复主键的值就会报错
* 应用运行时错误：这类错误范围很广，涵盖了代码中出现的几乎所有错误。可能的应用错误的情况如下：
    * 文件系统和权限：应用读取不存在的文件，或者读取没有权限的文件、或者写入一个不允许写入的文件，这些都会导致一个错误。应用读取的文件如果格式不正确也会报错，例如配置文件应该是ini的配置格式，而设置成了json格式就会报错。
    * 第三方应用：如果我们的应用程序耦合了其他第三方接口程序，例如应用程序发表文章之后自动调用接发微博的接口，所以这个接口必须正常运行才能完成我们发表一篇文章的功能。
* HTTP错误：这些错误是根据用户的请求出现的错误，最常见的就是404错误。虽然可能会出现很多不同的错误，但其中比较常见的错误还有401未授权错误(需要认证才能访问的资源)、403禁止错误(不允许用户访问的资源)和503错误(程序内部出错)。
* 操作系统出错：这类错误都是由于应用程序上的操作系统出现错误引起的，主要有操作系统的资源被分配完了，导致死机，还有操作系统的磁盘满了，导致无法写入，这样就会引起很多错误。
* 网络出错：指两方面的错误，一方面是用户请求应用程序的时候出现网络断开，这样就导致连接中断，这种错误不会造成应用程序的崩溃，但是会影响用户访问的效果；另一方面是应用程序读取其他网络上的数据，其他网络断开会导致读取失败，这种需要对应用程序做有效的测试，能够避免这类问题出现的情况下程序崩溃。

### 错误处理的目标

错误处理系统应该完成以下工作：

* 通知访问用户出现错误了：不论出现的是一个系统错误还是用户错误，用户都应当知道Web应用出了问题，用户的这次请求无法正确的完成了。例如，对于用户的错误请求，我们显示一个统一的错误页面(404.html)。出现系统错误时，我们通过自定义的错误页面显示系统暂时不可用之类的错误页面(error.html)。
* 记录错误：系统出现错误，一般就是我们调用函数的时候返回err不为nil的情况，可以使用前面小节介绍的日志系统记录到日志文件。如果是一些致命错误，则通过邮件通知系统管理员。一般404之类的错误不需要发送邮件，只需要记录到日志系统。
* 回滚当前的请求操作：如果一个用户请求过程中出现了一个服务器错误，那么已完成的操作需要回滚。下面来看一个例子：一个系统将用户递交的表单保存到数据库，并将这个数据递交到一个第三方服务器，但是第三方服务器挂了，这就导致一个错误，那么先前存储到数据库的表单数据应该删除(应告知无效)，而且应该通知用户系统出现错误了。
* 保证现有程序可运行可服务：我们知道没有人能保证程序一定能够一直正常的运行着，万一哪一天程序崩溃了，那么我们就需要记录错误，然后立刻让程序重新运行起来，让程序继续提供服务，然后再通知系统管理员，通过日志等找出问题

### 如何处理异常

很多其他语言中有try...catch关键词，用来捕获异常情况，但是其实很多错误都是可以预期发生的，而不需要异常处理，应该当做错误来处理，这也是为什么Go语言采用了函数返回错误的设计，这些函数不会panic.

但是还有一种情况，有一些操作几乎不可能失败，而且在一些特定的情况下也没有办法返回错误，也无法继续执行，这样情况就应该panic。

一个不可预期严重错误就会引起panic，在默认情况下它会杀掉进程，它允许一个正在运行这部分代码的goroutine从发生错误的panic中恢复运行，发生panic之后，这部分代码后面的函数和代码都不会继续执行，这是Go特意这样设计的，因为要区别于错误和异常，panic其实就是异常处理。如下代码，我们期望通过uid来获取User中的username信息，但是如果uid越界了就会抛出异常，这个时候如果我们没有recover机制，进程就会被杀死，从而导致程序不可服务。因此为了程序的健壮性，在一些地方需要建立recover机制。

上面介绍了错误和异常的区别，那么我们在开发程序的时候如何来设计呢？规则很简单：如果你定义的函数有可能失败，它就应该返回一个错误。当我调用其他package的函数时，如果这个函数实现的很好，我不需要担心它会panic，除非有真正的异常情况发生，即使那样也不应该是我去处理它。而panic和recover是针对自己开发package里面实现的逻辑，针对一些特殊情况来设计。